In [71]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import trange
import tqdm
import torchvision
import boto3
import io
import tempfile
import PIL
import time
from datetime import timedelta
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn import model_selection, metrics
import matplotlib.pyplot as plt
import itertools
import timm

In [72]:
# Make sure you're using cuda (GPU) by checking the hardware accelerator under Runtime -> Change runtime type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


In [73]:
class MyDatasetTest(Dataset):

    def __init__(self, df, transform=None):
        self.image_path = '/kaggle/input/cassava-leaf-disease-classification/test_images'
        self.labels = df
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.image_path + '/' + self.labels.iloc[idx]['image_id']

        # Read the image from the file path
        #image = Image.open(img_name)
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)       
        # Transform the image using self.transform
        if self.transform:
            image = self.transform(image=image)["image"]
        return image

In [74]:
# Credit: https://www.kaggle.com/code/aliabdin1/calculate-mean-std-of-images/notebook
mean = np.array([0.42984136, 0.49624753, 0.3129598 ])
std = np.array([0.21417203, 0.21910103, 0.19542212])

In [75]:
albu_transform_test = A.Compose(
        [A.Resize(height=384, width=384),
         A.Normalize(mean = mean, std = std),
         ToTensorV2()
        ])

In [76]:
test = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')

test_dataset = MyDatasetTest(test, transform=albu_transform_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, 
                         num_workers=2, pin_memory=True) 

In [77]:
class ResNext(nn.Module):
  def __init__(self, num_classes):
    super(ResNext, self).__init__()
    self.model = timm.create_model('resnext50_32x4d', pretrained=False)#torchvision.models.resnet34(weights='IMAGENET1K_V1')
    self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

  def forward(self, input):
    out = self.model(input)
    return out #F.softmax(out, dim=1)

In [78]:
model_ResNext = ResNext(5).to(device)
model_ResNext.load_state_dict(torch.load('/kaggle/input/cassavaweights/resnext50_tuning_epoch_20_acc_87.1807.pth'))

<All keys matched successfully>

In [79]:
class ResNet(nn.Module):
  def __init__(self, num_classes):
    super(ResNet, self).__init__()
    self.model = timm.create_model('resnet34d', pretrained=False)#torchvision.models.resnet34(weights='IMAGENET1K_V1')
    self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    self.softmax = nn.Softmax()

  def forward(self, input):
    out = self.model(input)
    #softmax = self.softmax(out)
    return out#softmax

In [80]:
model_ResNet = ResNet(5).to(device)
model_ResNet.load_state_dict(torch.load('/kaggle/input/cassavaweights/resnet34_acc_84.4237.pth'))

<All keys matched successfully>

In [81]:
class BCNN(nn.Module):
  def __init__(self, num_classes):
    super(BCNN, self).__init__()
    features = torchvision.models.resnet34()
    self.conv = nn.Sequential(*list(features.children())[:-2])
    self.fc = nn.Linear(512 * 512, num_classes)
    self.softmax = nn.Softmax()

  def forward(self, input):
    features = self.conv(input)
    features_size = features.size(2) * features.size(3)

    features = features.view(features.size(0), 512, features_size)

    features_T = torch.transpose(features, 1, 2)
    features = torch.bmm(features, features_T) / (features_size)

    features = features.view(features.size(0), 512 * 512)


    features = torch.sign(features) * torch.sqrt(torch.abs(features) + 1e-12)

    features = torch.nn.functional.normalize(features)


    out = self.fc(features)
    softmax = self.softmax(out)
    return softmax

In [82]:
model_BCNN = nn.DataParallel(BCNN(5)).cuda()
model_BCNN.load_state_dict(torch.load('/kaggle/input/cassavaweights/resnet34_tuning_epoch_39_acc_85.5607.pth'))

<All keys matched successfully>

In [83]:
class effnet(nn.Module):
  def __init__(self, num_classes):
    super(effnet, self).__init__()
    self.model = timm.create_model('tf_efficientnet_b4_ns', pretrained=False)
    self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)

  def forward(self, input):
    out = self.model(input)
    return out #F.softmax(out, dim=1)

In [84]:
model_effnet = effnet(5).to(device)
model_effnet.load_state_dict(torch.load('/kaggle/input/cassavaweights/effnetb4ns_aug_tuning_epoch_18_acc_86.324.pth'))

<All keys matched successfully>

In [85]:
class VGG19(nn.Module):
    def __init__(self, num_classes):
        super(VGG19, self).__init__()
        self.model = timm.create_model('vgg19', pretrained=False) #torchvision.models.resnet34(weights='IMAGENET1K_V1')
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(in_features=self.model.num_classes, out_features=num_classes)
        #self.softmax = nn.Softmax()
    def forward(self, input):
        out = self.model(input)
        out = self.drop(out)
        out = self.fc(out)
        #softmax = self.softmax(out)
        return out

In [86]:
model_vgg = VGG19(5).to(device)
model_vgg.load_state_dict(torch.load('/kaggle/input/cassavaweights/vgg19_acc_83.8162.pth'))

<All keys matched successfully>

In [87]:
class vit(nn.Module):
  def __init__(self, num_classes):
    super(vit, self).__init__()
    self.model = timm.create_model('vit_base_patch16_384', pretrained=False)
    self.model.head = nn.Linear(self.model.head.in_features, 5)

  def forward(self, input):
    out = self.model(input)
    return out #F.softmax(out, dim=1)

In [88]:
model_vit = vit(5).to(device)
model_vit.load_state_dict(torch.load('/kaggle/input/cassavaweights/vit_tuning_epoch_4_acc_88.2555.pth'))

<All keys matched successfully>

In [89]:
def inference(test_loader):
    probs = []
    for i, (images) in enumerate(test_loader):
        images = images.to(device)
        avg_preds = []
        model_ResNext.eval()
        with torch.no_grad():
            y_preds_resnext = model_ResNext(images)
            y_preds_resnext = y_preds_resnext.softmax(1)
        model_ResNet.eval()
        with torch.no_grad():
            y_preds_resnet = model_ResNet(images)
            y_preds_resnet = y_preds_resnet.softmax(1)
        model_BCNN.eval()
        with torch.no_grad():
            y_preds_BCNN = model_BCNN(images)
        model_effnet.eval()
        with torch.no_grad():
            y_preds_effnet = model_effnet(images)
            y_preds_effnet = y_preds_effnet.softmax(1)
        model_vgg.eval()
        with torch.no_grad():
            y_preds_vgg = model_vgg(images)
            y_preds_vgg = y_preds_vgg.softmax(1)
        with torch.no_grad():
            y_preds_vit = model_vit(images)
            y_preds_vit = y_preds_vit.softmax(1)
        y_preds = y_preds_vit#1*y_preds_effnet#+0.05*y_preds_resnet+0.4*y_preds_BCNN
        avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

In [90]:
predictions = inference(test_loader)
test['label'] = predictions.argmax(1)
test[['image_id', 'label']].to_csv('submission.csv', index=False)
test.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


,image_id,label
0,2216849948.jpg,4
